# Music Similarity Detection
In this notebook, we will detect similarity of music for a given dataset of songs. The dataset is 20 telugu songs in mp3 format. The MusiCNN model is used for evaluating similarity between the songs.
Evaluation metrics such as average precision@10, MAP@10, and MAP@50 will be computed across the entire dataset.

### Step 1: Import necessary libraries

In [2]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MultiLabelBinarizer
import plotly.express as px
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import json

2024-06-20 23:45:51.425285: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-20 23:45:51.463939: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-20 23:45:52.042527: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Step 2: Download model and define paths

Download the model and move to 'model' directory.
This step is not necessary if the model has already been downloaded.

In [2]:
!curl -SLO https://essentia.upf.edu/models/classifiers/genre_tzanetakis/genre_tzanetakis-musicnn-msd-1.json
!curl -SLO https://essentia.upf.edu/models/classifiers/genre_tzanetakis/genre_tzanetakis-musicnn-msd-1.pb

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0^C
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
 39 3166k   39 1237k    0     0   106k      0  0:00:29  0:00:11  0:00:18  111k

In [3]:
MUSICNN_SR = 16000
DATASET_DIR = '../dataset/mp3'

In [4]:
MODEL_NAME = '../model/genre_tzanetakis-musicnn-msd-1'
MODEL_JSON = f'{MODEL_NAME}.json'
MODEL_PB = f'{MODEL_NAME}.pb'

musicnn_metadata = json.load(open(MODEL_JSON, 'r'))
for k, v in musicnn_metadata.items():
    print('{}: {}'.format(k , v))

name: genre GTZAN
type: multi-class classifier
link: https://essentia.upf.edu/models/classifiers/genre_tzanetakis/genre_tzanetakis-musicnn-msd-1.pb
version: 1
description: classification of music by genre
author: Pablo Alonso
email: pablo.alonso@upf.edu
release_date: 2020-03-31
framework: tensorflow
framework_version: 1.15.0
classes: ['blu', 'cla', 'cou', 'dis', 'hip', 'jaz', 'met', 'pop', 'reg', 'roc']
model_types: ['frozen_model']
dataset: {'name': 'the GTZAN Genre Collection', 'citation': '@article{tzanetakis2002musical,\n  title={Musical genre classification of audio signals},\n  author={Tzanetakis, George and Cook, Perry},\n  journal={IEEE Transactions on speech and audio processing},\n  volume={10},\n  number={5},\n  pages={293--302},\n  year={2002},\n  publisher={IEEE}\n}', 'size': '1000 track excerpts, 100 per genre', 'metrics': {'5-fold_cross_validation_normalized_accuracy': 0.83}}
schema: {'inputs': [{'name': 'model/Placeholder', 'type': 'float', 'shape': [187, 96]}], 'output

We can observe the output of the penultimate dense layer is proposed as embeddings. 
We will use it to extract songs embeddings from our dataset.

### Step 3: Define genres for each song

In [5]:
genres = [
    ['Romantic', 'Melody'], # Allantha Doorala 0
    ['Romantic', 'Melody'], # Pranavalaya 1
    ['Romantic', 'Sad'], # Adiga Adiga 2
    ['Romantic', 'Upbeat'], # Guruvaram 3
    ['Romantic', 'Folk'],   # Ghal Ghal 4
    ['Romantic', 'Melody'], # Ninnila 5
    ['Mass', 'Inspirational'], # Hukum 6
    ['Melody'], # Chinnari Thalli 7
    ['Romantic', 'Melody'], # O Cheliya 8
    ['Folk', 'Dance'],      # Naatu Naatu 9 
    ['Folk', 'Dance'],      # Kurchi Madathapetti 10
    ['Romantic', 'Upbeat'], # Urike Urike 11
    ['Inspirational', 'Mass'], # Saahore Baahubali 12
    ['Romantic', 'Melody'], # Nenani Neevani 13
    ['Romantic', 'Melody'], # Manasavacha 14
    ['Romantic', 'Melody'],       # Ammaadi 15
    ['Romantic', 'Melody'], # Dheera Dheera 16
    ['Melody'], # Pedave Palikina 17 
    ['Romantic', 'Sad'],   # Oosupodu 18
    ['Romantic', 'Melody'] # Sirivennela 19
]

### Step 4: Encode genres

In [6]:
mlb = MultiLabelBinarizer()
genre_labels = mlb.fit_transform(genres)

### Step 5: Functions for loading and processing audio

In [7]:
def load_model(pb_file_path):
    with tf.io.gfile.GFile(pb_file_path, "rb") as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.compat.v1.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name="")
    return graph

def preprocess_audio(file_path, sample_rate=MUSICNN_SR):
    y, sr = librosa.load(file_path, sr=sample_rate)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=96, n_fft=2048, hop_length=512)
    log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    if log_mel_spec.shape[1] > 187:
        log_mel_spec = log_mel_spec[:, :187]
    else:
        pad_width = 187 - log_mel_spec.shape[1]
        log_mel_spec = np.pad(log_mel_spec, ((0, 0), (0, pad_width)), mode='constant')
    return log_mel_spec.T[np.newaxis, :, :]

def run_model(graph, input_data):
    input_tensor = graph.get_tensor_by_name('model/Placeholder:0')
    output_tensor = graph.get_tensor_by_name('model/dense/BiasAdd:0')
    with tf.compat.v1.Session(graph=graph) as sess:
        embeddings = sess.run(output_tensor, feed_dict={input_tensor: input_data})
    return embeddings

def extract_mean_embedding(filename):
    audio_data = preprocess_audio(filename)
    graph = load_model(MODEL_PB)
    embeddings = run_model(graph, audio_data)
    return embeddings

def process_dataset(dataset_dir):
    embeddings = []
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith('.mp3'):
                file_path = os.path.join(root, file)
                mean_embedding = extract_mean_embedding(file_path)
                embeddings.append((file_path, mean_embedding))
    return embeddings

### Step 6: Process dataset and extract embeddings

In [8]:
file_embeddings = process_dataset(DATASET_DIR)
embeddings = []
for file_path, embedding in file_embeddings:
    print(f'File path: {file_path}')
    embeddings.append(embedding)

embeddings = np.array(embeddings)
embeddings_matrix = np.vstack([emb for emb in embeddings]) 
similarities = cosine_similarity(embeddings_matrix)

2024-06-20 23:46:07.845000: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-20 23:46:07.898141: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-06-20 23:46:07.904834: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled


File path: ../dataset/mp3/Allantha Doorala.mp3
File path: ../dataset/mp3/Pranavalaya.mp3
File path: ../dataset/mp3/Adiga Adiga.mp3
File path: ../dataset/mp3/Guruvaram.mp3
File path: ../dataset/mp3/Ghal Ghal Ghal Ghal.mp3
File path: ../dataset/mp3/Ninnila.mp3
File path: ../dataset/mp3/Hukum.mp3
File path: ../dataset/mp3/Chinnari Thalli.mp3
File path: ../dataset/mp3/O Cheliya.mp3
File path: ../dataset/mp3/Nattu Nattu.mp3
File path: ../dataset/mp3/Kurchi Madathapetti.mp3
File path: ../dataset/mp3/Urike Urike.mp3
File path: ../dataset/mp3/Saahore Baahubali.mp3
File path: ../dataset/mp3/Nenani Neevani.mp3
File path: ../dataset/mp3/Manasavacha.mp3
File path: ../dataset/mp3/Ammaadi.mp3
File path: ../dataset/mp3/Dheera Dheera Dheera.mp3
File path: ../dataset/mp3/Pedave Palikina.mp3
File path: ../dataset/mp3/Oosupodu.mp3
File path: ../dataset/mp3/Sirivennela.mp3


In [9]:
print(similarities.shape)
similarities = similarities.round(5)
print(f'Min value: {similarities.min()}')
print(f'Max value: {similarities.max()}')
df_similarities = pd.DataFrame(similarities)
df_similarities.to_csv('../data/similarity_data.csv', index=False)
fig = px.imshow(similarities, color_continuous_scale='RdYlGn')
fig.show()

(20, 20)
Min value: 0.649370014667511
Max value: 1.0


### Step 7: Compute sorted indexes and similar songs matrix

In [10]:
sorted_indexes = np.argsort(similarities, axis=1)
sorted_indexes = np.fliplr(sorted_indexes)

def match_genres(genres_i, genres_j):
    return any(genre in genres_j for genre in genres_i)

similar_songs = np.zeros_like(sorted_indexes)

for i in range(sorted_indexes.shape[0]):
    genre_i = genres[i] 
    sorted_indexes_i = sorted_indexes[i, :]  
    similar_songs_i = np.array([1 if match_genres(genre_i, genres[j]) else 0 for j in sorted_indexes_i])
    similar_songs[i, :] = similar_songs_i


### Step 8: Evaluation metrics: AP@N and MAP@N

In [11]:
def precision_k(similar_items, k):
    similar_items_k = similar_items[:k]
    P = sum(similar_items_k) / k
    return P

def average_precision_N(similar_items, N):
    m = np.min([np.sum(similar_items), N])
    sum_vector = []
    for k in range(1, N+1):
        if k-1 < len(similar_items):
            sum_vector.append(precision_k(similar_items, k) * similar_items[k-1])
    AP = (1/m) * sum(sum_vector)
    return AP


### Step 9: Evaluate music similarity

In [12]:
N_range = [3, 5, 10, 13, 15]
AP_songs = []

for N in N_range:
    AP_by_N = []
    for i in range(similar_songs.shape[0]):
        AP_by_N.append(average_precision_N(similar_songs[i, :], N))
    AP_songs.append(AP_by_N)

genre_strings = [','.join(genre) for genre in genres]

df = pd.DataFrame(
    list(zip(
        [file_path for file_path in range(len(file_path))], 
        genre_strings,
        *AP_songs
    )),
    columns=['song_name', 'genre'] + [f'@{N}' for N in N_range]
)

df.head(20)

,song_name,genre,@3,@5,@10,@13,@15
0,0,"Romantic,Melody",1.000000,1.000000,0.866389,0.725625,0.681256
1,1,"Romantic,Melody",0.666667,0.710000,0.583333,0.621526,0.591037
2,2,"Romantic,Sad",1.000000,0.600000,0.513095,0.494922,0.505489
3,3,"Romantic,Upbeat",1.000000,1.000000,0.743492,0.629609,0.635658
4,4,"Romantic,Folk",0.555556,0.643333,0.625873,0.544378,0.568302
5,5,"Romantic,Melody",1.000000,1.000000,0.776389,0.719331,0.675801
6,6,"Mass,Inspirational",0.500000,0.500000,0.642857,0.642857,0.642857
7,7,Melody,0.666667,0.550000,0.332143,0.381785,0.427239
8,8,"Romantic,Melody",1.000000,0.800000,0.745437,0.690277,0.703954
9,9,"Folk,Dance",0.333333,0.333333,0.416667,0.507576,0.507576


### Step 10: Plotting AP@N histograms

In [13]:
fig = make_subplots(rows=2, cols=3, subplot_titles=("AP@2", "AP@3", "AP@5", "AP@10", "AP@15"))

fig.add_trace(go.Histogram(x=AP_songs[0]), row=1, col=1)
fig.add_trace(go.Histogram(x=AP_songs[1]), row=1, col=2)
fig.add_trace(go.Histogram(x=AP_songs[2]), row=1, col=3)
fig.add_trace(go.Histogram(x=AP_songs[3]), row=2, col=1)
fig.add_trace(go.Histogram(x=AP_songs[4]), row=2, col=2)

fig.update_layout(height=500, 
                  width=1000, 
                  showlegend=False,
                  title_text="AP@N histograms")

fig.show()

### Step 11: Compute mAP@N for each genre and for the whole dataset

In [14]:
numeric_columns = [f'@{N}' for N in N_range]

mAP_by_genre = df.groupby('genre')[numeric_columns].mean()
mAP_dataset = df[numeric_columns].mean().to_frame().rename(columns={0: 'mAP'})

print('Mean average precision for each genre: ')
print(mAP_by_genre)
print('Mean average precision for entire dataset: ')
print(mAP_dataset)


fig = go.Figure()
for genre in df['genre'].unique():
    fig.add_trace(go.Scatter(y=mAP_by_genre.loc[genre], x=N_range, mode='lines+markers', name=genre))
fig.add_trace(go.Scatter(y=mAP_dataset['mAP'], x=N_range, mode='lines+markers', name='mean', line=dict(width=4, dash='dash')))

fig.update_layout(title='MAP@N by genre', xaxis_title='N', yaxis_title='MAP@N')
fig.show()

Mean average precision for each genre: 
                          @3        @5       @10       @13       @15
genre                                                               
Folk,Dance          0.333333  0.333333  0.408333  0.453788  0.453788
Inspirational,Mass  0.500000  0.500000  0.642857  0.642857  0.642857
Mass,Inspirational  0.500000  0.500000  0.642857  0.642857  0.642857
Melody              0.833333  0.655000  0.484345  0.480232  0.528934
Romantic,Folk       0.555556  0.643333  0.625873  0.544378  0.568302
Romantic,Melody     0.888889  0.858889  0.720983  0.686000  0.664692
Romantic,Sad        1.000000  0.700000  0.525595  0.531164  0.567884
Romantic,Upbeat     0.833333  0.775000  0.646627  0.578520  0.614408
Mean average precision for entire dataset: 
          mAP
@3   0.777778
@5   0.715000
@10  0.626512
@13  0.604575
@15  0.608314


Saving the similarities to a csv file

In [15]:
# df_similarities = pd.DataFrame(similarities)
# df.to_csv('../data/genre_similarity.csv',index=False)
mAP_by_genre.to_csv('../data/map_by_genre.csv')

In [20]:
mAP_dataset = mAP_dataset.transpose()
mAP_dataset = mAP_dataset.rename(index={'mAP':'All Combined'})
print(mAP_dataset)

                    @3     @5       @10       @13       @15
All Combined  0.777778  0.715  0.626512  0.604575  0.608314


In [21]:
mAP_dataset.to_csv('../data/demucs_similarity.csv', mode='a', header=False)

Saving the song names

In [24]:
genres = [
     'Allantha Doorala' 
    , 'Pranavalaya' 
    , 'Adiga Adiga' 
    , 'Guruvaram' 
    , 'Ghal Ghal' 
    , 'Ninnila' 
    , 'Hukum' 
    , 'Chinnari Thalli' 
    , 'O Cheliya' 
    , 'Naatu Naatu'  
    , 'Kurchi Madathapetti' 
    , 'Urike Urike' 
    , 'Saahore Baahubali' 
    , 'Nenani Neevani'
    , 'Manasavacha'
    , 'Ammaadi' 
    , 'Dheera Dheera' 
    , 'Pedave Palikina'  
    , 'Oosupodu' 
    , 'Sirivennela' 
]

print(len(genres))

20


In [32]:
import pandas as pd

data = pd.read_csv('../data/genre_similarity.csv')
data = pd.DataFrame(data)
data['song_name'] = genres
data = data.rename(columns={'song_name':'Song Name','genre':'Genre'})
data.to_csv('../data/genre_similarity.csv',index=False)